In [1]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Testing groot to many predicates


In [2]:
predicates = {'source': ['workedunder'],
              'target': ['samevenue']}

In [3]:
bk = json.loads(open('src/experiments/kb.txt').readline())


In [4]:
final_results = {}
source_pred = predicates['source']
target_pred = predicates['target']

for pred in zip(source_pred, target_pred):
    final_results[f'{pred[0]}->{pred[1]}'] = ''
    final_results[f'inf_res:{pred[0]}->{pred[1]}'] = ''
    final_results[f'groot_test:{pred[0]}->{pred[1]}'] = ''
    final_results[f'tree_test:{pred[0]}->{pred[1]}'] = ''

In [5]:
kb_source = bk['imdb']
kb_target = bk['cora']

In [6]:
pred_target = create_pred_target(kb_target)

In [9]:
for source, target in zip(source_pred, target_pred):
    print(f'Iniciando transferência {source}->{target}')

    imdb_dataset = datasets.load('imdb', kb_source, target=source, seed=441773) #facts, pos, neg
    cora_dataset = datasets.load('cora', kb_target, target=target, seed=441773) #facts, pos, neg
    
#     target = f'recursion_{target}'

    train_facts_source, train_pos_source, train_neg_source = get_train_division(imdb_dataset)
    background_knowledge_src = boostsrl.modes(kb_source, [source], useStdLogicVariables=False, 
                                          maxTreeDepth=3, nodeSize=2, numOfClauses=8)
    model_src = boostsrl.train(background_knowledge_src, train_pos_source, train_neg_source, train_facts_source, trees=10)
    
    structured_src, new_src_struct = create_structured_trees(model_src)
    
    
    train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(cora_dataset)
    
    start = time.time()
    final_results[f'{source}->{target}'] = genetic(new_src_struct, target, source, train_pos, 
                                             train_neg, train_facts, kb_source, kb_target, pred_target,
                                              NUM_GEN=14, pop_size=30)
    print("FIM DO GROOT: ", time.time()-start)
    
    tr_train_pos = []
    tr_train_neg = []
    tr_train_facts = []

    for i in range(0, len(train_pos)):
        tr_train_pos.extend(train_pos[i])
        tr_train_neg.extend(train_neg[i])
        tr_train_facts.extend(train_facts[i])
        
    train = [tr_train_pos, tr_train_neg, tr_train_facts]
    test = [test_pos, test_neg, test_facts]
    
    individual = get_best_individual(final_results[f'{source}->{target}'][0].population)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'cora')
    
    final_results[f'groot_test:{source}->{target}'], model = test_refine_transfer(kb_target, target, refine, transfer, train, test)
    
    final_results[f'inf_res:{source}->{target}'] = model.inference_results(target)
    
    final_results[f'tree_test:{source}->{target}'] = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)
    
    save_results(final_results, source, target, 'imdb', 'cora')
    
    print(f"Finalizando transferência {source}->{target} com tempo {time.time()-start} s")

Iniciando transferência workedunder->samevenue


/usr/local/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


workedunder samevenue
workedunder(person,person). samevenue(venue,venue).
['0;;samevenue(A, B) :- samevenue(C, A), samevenue(C, B).;true;true'] ['0;;workedunder(A, B) :- movie(C, A), movie(C, B).;true;true']
workedunder(person,person). samevenue(venue,venue).
movie(movie,person). samevenue(venue,venue).
movie(movie,person). samevenue(venue,venue).
['0;;samevenue(A, B) :- samebib(C, A), sameauthor(C, B).;true;true'] ['0;;workedunder(A, B) :- movie(C, A), movie(C, B).;true;true']
workedunder(person,person). samevenue(venue,venue).
movie(movie,person). samebib(class,class).
movie(movie,person). sameauthor(author,author).
['1;;samevenue(A, B) :- samevenue(C, A), samevenue(C, B).;true;true'] ['1;;workedunder(A, B) :- movie(C, A), movie(C, B).;true;true']
workedunder(person,person). samevenue(venue,venue).
movie(movie,person). samevenue(venue,venue).
movie(movie,person). samevenue(venue,venue).
['1;;samevenue(A, B) :- haswordvenue(C, A), sametitle(C, B).;true;true'] ['1;;workedunder(A, B) :-

KeyboardInterrupt: 